# WiFinder

#### About WiFinder

WiFinder is a Jupyter-based project that ingests iOS WiFi database files (i.e., "consolidated.db") and outputs a map of locations with relevant metadata.

WiFinder will also ingest lists of IP addresses and output a map of locations with relevant metada.

#### Dependencies

In [ ]:
!pip freeze

#### Sources

* https://stackoverflow.com/questions/29216889/slicing-a-dictionary
* https://developer.apple.com/documentation/corefoundation/cfabsolutetime
* https://stackoverflow.com/questions/16901279/convert-mac-timestamps-with-python
* https://sqliteviewer.app/
* https://www.sqlitetutorial.net/sqlite-python/sqlite-python-select/
* https://pypi.org/project/mac-vendor-lookup/
* https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/
* https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6
* https://fontawesome.com/icons
* https://pynative.com/python-json-load-and-loads-to-parse-json/
* https://www.freecodecamp.org/news/how-to-get-location-information-of-ip-address-using-python/
* http://sleuthkit.org/autopsy/docs/user-docs/4.19.3/ds_page.html#ds_log

## Boilerplate

### Setup

In [1]:
from IPython.display import display, clear_output
from mac_vendor_lookup import AsyncMacLookup
from timezonefinder import TimezoneFinder
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from sqlite3 import connect
from pathlib import Path

import ipywidgets as widgets

import datetime
import requests
import codecs
import folium
import json as js
import time
import os

from wifinder import Display, DBDisplay, IPDisplay

### Classes

#### Display Superclass

In [ ]:
# display base class
class Display(object):
    def __init__(self,
                 accepted_filetypes: str,
                 button_desc: str,
                 button_tooltip: str,
                 button_icon: str
                 ):
        self.output = widgets.Output()
        self.data = {}

        # create upload tab
        uploader = widgets.FileUpload(accept=accepted_filetypes,
                                      multiple=False)
        button_resolve = widgets.Button(description=button_desc,
                                        disabled=False,
                                        tooltip=button_tooltip,
                                        button_style='info',
                                        icon=button_icon)
        button_resolve.on_click(self.button_resolve_pressed)
        button_save = widgets.Button(description="Save Results",
                                     disabled=False,
                                     tooltip="Save Metadata to JSON",
                                     button_style='info',
                                     icon='download')
        button_save.on_click(self.button_save_pressed)
        self.upload = widgets.HBox([uploader, button_resolve, button_save])

        # create file options tab
        check_unique = widgets.Checkbox(value=True,
                                        description='Include duplicate entries',
                                        disabled=False,
                                        indent=False)
        check_zeroes = widgets.Checkbox(value=False,
                                        description='Include unresolved entries',
                                        disabled=False,
                                        indent=False)
        int_max = widgets.IntText(value=None,
                                  description="Max entries",
                                  disabled=False,
                                  layout=widgets.Layout(width='initial'))
        self.options = widgets.HBox([check_unique, check_zeroes, int_max])

        # create map options tab
        dropdown_map = widgets.Dropdown(options=['OpenStreetMap', 'Stamen Terrain', 'Stamen Toner',
                                                 'Stamen Watercolor', 'CartoDB positron', 'CartoDB dark_matter'],
                                        value='OpenStreetMap',
                                        description='Map type',
                                        disabled=False)
        self.map_options = widgets.HBox([dropdown_map])

        # create struct object (contains all other objects)
        self.struct = widgets.Tab()
        self.struct.children = [self.upload, self.options, self.map_options]
        self.struct.titles = ["Upload File", " File Options", "Map Options"]
        self.display(self.struct)

    # ====================
    # Interaction Methods
    # ====================

    # react to resolve button pressed
    def button_resolve_pressed(self, button):
        return None

    # react to JSON save button pressed
    def button_save_pressed(self, button):
        try:
            json_dict = dict([(str(key), self.data[key]) for key in list(self.data.keys())])
            json_obj = js.dumps(json_dict, indent=4)
            with open("./json_{}.json".format(self.upload.children[0].value[0].name), "w", encoding="utf-8") as jf:
                jf.write(json_obj)
            self.upload.children[2].button_style = 'success'
        except:
            self.upload.children[2].button_style = 'warning'

    # ====================
    # Visualization Methods
    # ====================

    # create temporary progress bar
    def get_progress(self, val: int):
        return widgets.IntProgress(
            value=0,
            min=0,
            max=val,
            description="Processing",
            bar_style="success",
            style={"bar_color": "yellow"},
            orientation="horizontal")

    def display(self, obj):
        clear_output()
        display(obj)

#### DBDisplay Class

In [ ]:
# populate display for DB upload
class DBDisplay(Display):
    def __init__(self):
        super().__init__(".db, .json", "Connect", "Click to Connect to DB", 'database')

    # ====================
    # Visualization Methods
    # ====================

    def get_struct(self):
        return self.data[list(self.data.keys())[0]]

    # ====================
    # Interaction Methods
    # ====================

    # react to resolve button pressed
    def button_resolve_pressed(self, button):
        try:
            self.db_connect()
            self.upload.children[1].button_style = 'success'
            self.upload.children[2].button_style = 'info'
        except:
            self.upload.children[1].button_style = 'warning'

    # ====================
    # Background Methods
    # ====================

    # create temporary db file
    def tmp_save(self):
        if len(self.upload.children[0].value) > 0:
            os.makedirs("./.tmp", exist_ok=True)
            self.tmp_file = Path(os.path.join("./.tmp", self.upload.children[0].value[0].name)).expanduser().resolve()

            with open(self.tmp_file, "wb") as fp:
                fp.write(self.upload.children[0].value[0].content)
        else:
            return False
        return True

    # delete temporary db file
    def tmp_del(self):
        os.remove(self.tmp_file)

    # ====================
    # Metadata Methods
    # ====================

    # connect to and read database
    def db_connect(self):
        # handle .db/.json upload
        if len(self.upload.children[0].value) > 0:
            ext = self.upload.children[0].value[0].name.split(".")[-1]
            # if .json is uploaded, read from memory and block re-parsing later
            if ext == "json":
                data_raw = js.loads(self.upload.children[0].value[0].content.tobytes())
                data = {}

                for coord_raw in data_raw:
                    coord = tuple([float(k) for k in coord_raw.strip("()").split(", ")])

                    if self.options.children[1].value is False:
                        if coord[0] == 0.0 and coord[1] == 0.0:
                            continue

                    # if duplicate entries is true, always add MAC entry
                    # if duplicate entries is false, only add one MAC entry
                    data[coord] = {'db_meta': [],
                                   'loc_meta': {}}

                    data[coord]['db_meta'] = data_raw[coord_raw]['db_meta'] if sui.options.children[
                                                                                   0].value is True else \
                    data_raw[coord_raw]['db_meta'][0]
                    data[coord]['loc_meta'] = data_raw[coord_raw]['loc_meta']

            # if .db is uploaded, save to temp database and connect
            elif ext == "db":
                if not self.tmp_save():
                    # break and pass to error handler if saving cannot occur
                    assert False

                # read sql database
                conn = connect(self.tmp_file)
                cur = conn.cursor()
                cur.execute("SELECT * from WifiLocation")
                rows = cur.fetchall()

                self.tmp_del()

                data = {}
                # format: (lat, lon): [MAC address, 802.11 TSFT Timestamp]
                for row in [r[0:4] for r in rows]:
                    coord = row[2:4]
                    if coord not in data:
                        if self.options.children[1].value is False and (coord[0] == 0.0 and coord[1] == 0.0):
                            continue

                    data[coord] = {'db_meta': [],
                                   'loc_meta': {}
                                   }

                    # if duplicate entries is true, always add MAC entry
                    entry = {'mac': row[0],
                             'manf': "",
                             'cfabsolute': row[1],
                             'timestamp': ""
                             }
                    if self.options.children[0].value is True:
                        data[coord]["db_meta"].append(entry)
                    # if duplicate entries is false, only add one MAC entry
                    elif len(data[coord]["db_meta"]) == 0:
                        data[coord]["db_meta"].append(entry)

            # trim from value cap option
            if self.options.children[2].value > 0:
                data = dict([(key, data[key]) for key in list(data.keys())[0:int(self.options.children[2].value)]])

            self.data = data

    # from metadata, get map object
    def build_map(self):
        self.marker_cluster = MarkerCluster()

        for coord in self.data:
            title = """
            <b>Location:</b><br>
            {}<hr>
            <b>Coordinates:</b><br>
            {} {}<hr>
            <b>MAC Address:</b><br>
            {} <hr>
            <b>Timestamp:</b><br>
            {}"""

            tool = "{} {}".format(coord[0], coord[1])

            if self.options.children[0].value is True:
                # populate by MAC occurance
                for entry in self.data[coord]['db_meta']:
                    title = title.format(self.data[coord]['loc_meta']['display_name'],  # location
                                         coord[0], coord[1],  # coordinates
                                         "{} ({})".format(entry['mac'], entry['manf']) if entry['manf'] != "" else
                                         entry['mac'],  # MAC address
                                         entry['timestamp'])  # timestamp
                    # create point
                    folium.Marker(location=(coord[0], coord[1]),
                                  popup=title,
                                  tooltip=tool
                                  ).add_to(self.marker_cluster)

            else:
                # populate by coord occurance
                entry = self.data[coord]['db_meta'][0]
                title = title.format(self.data[coord]['loc_meta']['display_name'],  # location
                                     coord[0], coord[1],
                                     "{} ({})".format(entry['mac'], entry['manf']) if entry['manf'] != "" else entry[
                                         'mac'],
                                     entry['timestamp'])
                # create point
                folium.Marker(location=(coord[0],
                                        coord[1]),
                              popup=title,
                              tooltip=tool
                              ).add_to(self.marker_cluster)

    # update map from options and display
    def get_map(self):
        start_coords = list(self.data.keys())[0]
        self.map = folium.Map(location=[start_coords[0],
                                        start_coords[1]],
                              zoom_start=2,
                              tiles=self.map_options.children[0].value)
        self.marker_cluster.add_to(self.map)

        self.display(self.map)

#### IPDisplay Class

In [ ]:
# populate display for IP uploads
class IPDisplay(Display):
    def __init__(self):
        super().__init__(".ip, .txt", "Search", "Click to Search IPs", 'wifi')
        self.map_options.children[0].value = "CartoDB positron"

    # ====================
    # Interaction Methods
    # ====================

    # react to resolve button pressed
    def button_resolve_pressed(self, button):
        try:
            if len(self.upload.children[0].value) > 0:
                ips = self.upload.children[0].value[0].content.tobytes().decode().replace('\r', '').split('\n')

                for ip in ips:
                    self.data[ip] = {}
            self.upload.children[1].button_style = 'success'
            self.upload.children[2].button_style = 'info'
        except:
            self.upload.children[1].button_style = 'warning'

    # ====================
    # Collection Methods
    # ====================

    # query ipapi API for metadata
    def get_loc(self, ip_addr):
        try:
            return requests.get(f'https://ipapi.co/{ip_addr}/json/').json()
        except:
            return None

    # format returned metadata to data struct
    def format_meta(self, ip_addr, meta):
        if meta is None:
            self.data[ip_addr] = None
        elif 'error' in meta:
            self.data[ip_addr] = None
        elif meta['latitude'] is None or meta['longitude'] is None:
            self.data[ip_addr] = None
        else:
            self.data[ip_addr] = meta
            self.data[ip_addr]['coord'] = [meta['latitude'], meta['longitude']]

    # ====================
    # Metadata Methods
    # ====================

    # from metadata, get map object
    def build_map(self):
        self.marker_cluster = MarkerCluster()
        for ip in self.data:
            # for simplicity,
            if self.data[ip] is None:
                continue

            title = """
            <b>IP Address</b><br>
            {}<hr>
            <b>Organization</b><br>
            {}<hr>
            <b>Network</b><br>
            {}<hr>
            <b>Location</b><br>
            {}<hr>
            <b>Timezone</b><br>
            {}"""

            tool = ip

            # populate map by IP address
            title = title.format(self.data[ip]['ip'],
                                 self.data[ip]['org'],
                                 self.data[ip]['network'],
                                 "{}, {}, {} {}".format(self.data[ip]['city'],
                                                        self.data[ip]['region'],
                                                        self.data[ip]['country_name'],
                                                        "({})".format(self.data[ip]['postal']) if self.data[ip][
                                                                                                      'postal'] is not None else ""
                                                        ),
                                 self.data[ip]['timezone']
                                 )
            # add marker to marker cluster
            folium.Marker(location=(self.data[ip]['coord'][0],
                                    self.data[ip]['coord'][1]),
                          popup=title,
                          tooltip=tool
                          ).add_to(self.marker_cluster)

    # update map from options and display
    def get_map(self):
        start_coords = self.data[list(self.data.keys())[0]]['coord']
        self.map = folium.Map(location=[start_coords[0],
                                        start_coords[1]],
                              zoom_start=2,
                              tiles=self.map_options.children[0].value)
        self.marker_cluster.add_to(self.map)

        self.display(self.map)

## Display Superclass

In [ ]:
dsp = Display("all", "Button", "Button", "button")

## DBDisplay

#### 1. Data Ingest

In [ ]:
sui = DBDisplay()

#### 2. Metadata Consumption

In [ ]:
# create loading display
prog = sui.get_progress(len(sui.data))
sui.display(prog)

cfabsolute = datetime.datetime.strptime("01-01-2001", "%m-%d-%Y")
zone_finder = TimezoneFinder()
mac = AsyncMacLookup()
geolocator = Nominatim(user_agent="geoapiExercises")

# parse metadata and perform lookups
for coord in sui.data:
    for num, entry in enumerate(sui.data[coord]['db_meta']):
        
        # get timestamp from 802.11 CFAbsoluteTime timestamp
        timestamp = (cfabsolute + datetime.timedelta(seconds=entry['cfabsolute'])).strftime("%a, %d %b %Y %H:%M:%S")
        timezone = zone_finder.timezone_at(lat=coord[0], lng=coord[1])
        sui.data[coord]['db_meta'][num]['timestamp'] = ("{} ({})".format(timestamp, timezone))

        # get MAC address manufacturers
        try:
            id = await mac.lookup(entry['mac'])
            sui.data[coord]["db_meta"][num]['manf'] = id
        except:
            sui.data[coord]["db_meta"][num]['manf'] = ""
            
    # get addresses from coordinates
    json = geolocator.reverse("{},{}".format(coord[0], coord[1])).raw
    sui.data[coord]["loc_meta"] = json
    
    prog.value += 1
prog.style={"bar_color": "green"}

#### 3. Map Plotting

In [ ]:
# format markers for map
sui.build_map()

# create and display map
sui.get_map()

## IPDisplay

#### 1. Data Ingest

In [ ]:
ipd = IPDisplay()

#### 2. Metadata Consumption

In [ ]:
# create loading display
ip_prog = ipd.get_progress(len(ipd.data))
ipd.display(ip_prog)

# parse metadata and perform lookups
for ip in ipd.data:
    meta = ipd.get_loc(ip)
    ipd.format_meta(ip, meta)
    ip_prog.value += 1
ip_prog.style={"bar_color": "green"}

# format markers for map
ipd.build_map()

#### 3. Map Plotting

In [ ]:
ipd.get_map()